# 3. プロンプトエンジニアリング


In [4]:
# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

import os
from dotenv import load_dotenv

current_working_directory = os.getcwd()
dotenv_path = os.path.join(current_working_directory, 'rag_ai_agent_book', '.env')

print(f"現在の作業ディレクトリ: {current_working_directory}")
print(f"参照する.envパス: {dotenv_path}")

load_dotenv(dotenv_path=dotenv_path)
# 環境変数がロードされたか確認し、必要であればos.environに設定
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    print("OPENAI_API_KEY 環境変数が設定されていません。")
    print("rag_ai_agent_book/ディレクトリに.envファイルを作成し、OpenAI APIキーを設定してください。")
    print("例: OPENAI_API_KEY=\"sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\"")
else:
    os.environ["OPENAI_API_KEY"] = api_key # 他のライブラリがos.environから直接読み込むことを期待する場合のために設定
    print("OPENAI_API_KEY が正常に設定されました。")


現在の作業ディレクトリ: /Users/kenichi/Projects
参照する.envパス: /Users/kenichi/Projects/rag_ai_agent_book/.env
OPENAI_API_KEY が正常に設定されました。


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
#!pip install openai==1.40.6 httpx==0.27.2

## 3.2. プロンプトエンジニアリングとは


In [5]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)

プロンプトエンジニアリングとは、特に人工知能（AI）や自然言語処理（NLP）モデルを活用する際に、モデルに対して効果的な入力（プロンプト）を設計・調整する手法や技術のことを指します。これにより、モデルから得られる出力の質や精度を向上させることを目的としています。

プロンプトエンジニアリングの主な要素には以下が含まれます：

1. **プロンプトの設計**: モデルに与える指示や質問をどのように構築するか。具体的で明確なプロンプトは、より良い応答を引き出す可能性があります。

2. **コンテキストの提供**: モデルに必要な背景情報や具体的な例を与えることで、より正確な結果を得るためのサポートをします。

3. **試行錯誤**: 様々なプロンプトをテストし、どの表現が最も良い結果をもたらすかを見極めるプロセスです。

4. **フィードバックの活用**: モデルの応答を分析し、それに基づいてプロンプトを改善することも重要です。

プロンプトエンジニアリングは、チャットボット、テキスト生成、質問応答システムなど、多くのAIアプリケーションにおいて重要な役割を果たしています。効果的なプロンプトを設計することで、ユーザーのニーズにより適切に応えることが可能になります。


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "質問に100文字程度で答えてください。"},
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)

## 3.3. プロンプトの構成要素の基本


### プロンプトのテンプレート化


In [6]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            # formatはPythonの文字列メソッドで、波かっこ{}の部分に値を埋め込むために使います。
            # 例えば、promptが「料理名: {dish}」なら、prompt.format(dish="カレー")で「料理名: カレー」になります。
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

カレーのレシピをご紹介します。簡単に作れる基本的なチキンカレーのレシピを考えてみました。

### 材料（4人分）

- 鶏むね肉または鶏もも肉：400g（1口大にカット）
- 玉ねぎ：2個（薄切り）
- ジャガイモ：2個（1口大にカット）
- ニンジン：1本（1口大にカット）
- カレールー：1箱（約200～250g）
- サラダ油：大さじ2
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- 水：800ml
- 塩：適量
- 黒胡椒：適量
- garnish（オプション）：パセリや刻みネギ、ヨーグルトなど

### 手順

1. **下準備**：
   - 鶏肉に塩と黒胡椒をふりかけて、しばらく置いておきます。

2. **玉ねぎを炒める**：
   - 大きな鍋にサラダ油を熱し、中火で薄切りにした玉ねぎを加えます。玉ねぎが透明になるまで炒めます。

3. **にんにくと生姜を追加**：
   - 玉ねぎが透明になったら、みじん切りにしたにんにくと生姜を加え、香りが立つまで炒めます。

4. **鶏肉を加える**：
   - 鶏肉を鍋に入れ、表面が白くなるまで炒めます。

5. **野菜を加える**：
   - ジャガイモとニンジンを加え、軽く炒めて全体を混ぜます。

6. **水を加える**：
   - 水を加えて、鍋を強火にします。煮立ったら、アクをすくい取り、弱火にして蓋をして約15分煮込みます。

7. **カレールーを加える**：
   - カレールーを割り入れ、よく混ぜて溶かします。さらに10分ほど煮込み、味がなじんだら火を止めます。

8. **調整**：
   - 必要に応じて、塩や黒胡椒で味を調整します。

9. **盛り付け**：
   - お皿にご飯を盛り、その上にカレーをかけます。オプションでパセリや刻みネギ、ヨーグルトを添えて完成です。

### アドバイス
- お好みで、他の野菜（例えばピーマンやカリフラワー）や豆類を加えることで、アレンジが可能です。
- 辛さを調整するために、チリパウダーやカイエンペッパーを加えることもできます。

おいしいカレーを楽しんでください！


In [8]:
def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "ユーザーが入力した料理のレシピを考えてください。"},
            {"role": "user", "content": f"{dish}"},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

もちろんです！ここにおいしいカレーのレシピをご紹介します。

### 【材料】（4人分）
- 鶏もも肉（または豚肉、牛肉）: 500g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレー粉: 2～3 tablespoons （お好みで調整）
- ブイヨン（または水＋顆粒だし）: 500ml
- サラダ油: 大さじ2
- 塩: 適量
- こしょう: 適量
- 砂糖: 小さじ1
- 醤油: 小さじ1
- ココナッツミルク（オプション）: 200ml

### 【作り方】
1. **材料の下ごしらえ**
   - 鶏肉は一口大に切り、塩、こしょうで下味をつける。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。

2. **炒める**
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒める。色が透明になるまで炒める。
   - 次に鶏肉を加え、表面が白くなるまで炒める。

3. **野菜を加える**
   - にんじんとじゃがいもを加え、全体がよく混ざるように炒める。

4. **煮込む**
   - ブイヨン（または水＋顆粒だし）を加え、煮立ったらアクを取り除く。
   - 中火にして10分ほど煮る。

5. **カレー粉を加える**
   - カレー粉、砂糖、醤油を加え、弱火でさらに20～30分煮込む。時々かき混ぜて、焦げ付かないように注意する。

6. **仕上げ**
   - （オプション）最後にココナッツミルクを加え、5分ほど煮て全体がなじんだら、味見して塩で調整する。

7. **盛り付け**
   - ご飯やナンと一緒に皿に盛りつけて、お好みでパセリや香菜を散らして完成！

### 【ポイント】
- お好みで、他の野菜（ピーマン、茄子など）を加えてもおいしいです。
- 辛さが苦手な方は、カレー粉の量を調整してください。

このカレーを家族や友人と一緒に楽しんでくださいね！


### 出力形式を指定する


In [9]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)

```json
{
  "材料": ["米", "鶏肉", "玉ねぎ", "にんじん", "じゃがいも", "カレールー", "水", "塩", "胡椒"],
  "手順": [
    "米を洗って炊飯器で炊く。",
    "鶏肉を一口大に切り、塩と胡椒で下味をつける。",
    "玉ねぎ、にんじん、じゃがいもを適当な大きさに切る。",
    "鍋に油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加えて表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加えて、さらに炒める。",
    "水を追加し、沸騰させたらアクを取り除く。",
    "弱火にして15分ほど煮込む。",
    "カレールーを加えて全体をよく混ぜ、さらに10分煮込む。",
    "炊き上がった米を皿に盛り、カレーをかけて完成。"
  ]
}
```


## 3.4. プロンプトエンジニアリングの定番の手法


### Zero-shot プロンプティング


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。",
        },
        {
            "role": "user",
            "content": "ChatGPTはとても便利だ",
        },
    ],
)
print(response.choices[0].message.content)

### Few-shot プロンプティング


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "AIの進化はすごい"},
        {"role": "assistant", "content": "true"},
        {"role": "user", "content": "今日は良い天気だ"},
        {"role": "assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

### （コラム）Few-shot プロンプティングのその他の形式


In [ ]:
prompt = """\
入力がAIに関係するか回答してください。

Q: AIの進化はすごい
A: true
Q: 今日は良い天気だ
A: false
Q: ChatGPTはとても便利だ
A:
"""

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
)
print(response.choices[0].text)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "system", "name": "example_user", "content": "AIの進化はすごい"},
        {"role": "system", "name": "example_assistant", "content": "true"},
        {"role": "system", "name": "example_user", "content": "今日は良い天気だ"},
        {"role": "system", "name": "example_assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

### Zero-shot Chain-of-Thought プロンプティング


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "回答だけ一言で出力してください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "ステップバイステップで考えてください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)